In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

In [ ]:
!wget https://dlcdn.apache.org/spark/spark-3.2.1/spark-3.2.1-bin-hadoop3.2.tgz

--2022-03-02 06:52:44--  https://dlcdn.apache.org/spark/spark-3.2.1/spark-3.2.1-bin-hadoop3.2.tgz
Resolving dlcdn.apache.org (dlcdn.apache.org)... 151.101.2.132, 2a04:4e42::644
Connecting to dlcdn.apache.org (dlcdn.apache.org)|151.101.2.132|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 300971569 (287M) [application/x-gzip]
Saving to: ‘spark-3.2.1-bin-hadoop3.2.tgz’

spark-3.2.1-bin-had 100%[===================>] 287.03M   224MB/s    in 1.3s    

2022-03-02 06:52:45 (224 MB/s) - ‘spark-3.2.1-bin-hadoop3.2.tgz’ saved [300971569/300971569]



In [ ]:
!tar -xzf /content/spark-3.2.1-bin-hadoop3.2.tgz

In [ ]:
!pip install -q findspark

In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.2.1-bin-hadoop3.2"

In [ ]:
import findspark

In [ ]:
findspark.init("/content/spark-3.2.1-bin-hadoop3.2/")

In [ ]:
findspark.find()

'/content/spark-3.2.1-bin-hadoop3.2/'

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
spark= SparkSession \
       .builder \
       .appName("Our First Spark example") \
       .getOrCreate()

In [ ]:
schema = StructType([
                     StructField("id", IntegerType(), True),
                     StructField("name", StringType(), True),
                     StructField("json", StringType(), True)
])

In [ ]:
df = spark.read.load('/content/drive/MyDrive/Data/book1.csv', format='csv', sep=';', schema=schema, header=True)

In [ ]:
# sch = StructType([
#                          StructField("a", MapType(StringType(), StringType())),
#                          StructField("b", MapType(StringType(), StringType())),
#                          StructField("c", MapType(StringType(), StringType())),
#                     ])

# sch =  StructType([StructField("data", MapType(StringType(), IntegerType()))])

sch =  ArrayType(MapType(StringType(), IntegerType()))

In [ ]:
df_01 = df.select(from_json(df.json, sch).alias("json"))

In [ ]:
df_01.printSchema()

root
 |-- json: array (nullable = true)
 |    |-- element: map (containsNull = true)
 |    |    |-- key: string
 |    |    |-- value: integer (valueContainsNull = true)



In [ ]:
df_01.show(5, False)

+----------------------------------------------------+
|json                                                |
+----------------------------------------------------+
|[{a -> 1, b -> 2, c -> 3}, {a -> 1, b -> 2, c -> 3}]|
|[{a -> 1, b -> 2, c -> 3}]                          |
|[]                                                  |
+----------------------------------------------------+



In [ ]:
df_01.select(to_json(df_01.json)).show(5, False)

+-----------------------------------------+
|to_json(json)                            |
+-----------------------------------------+
|[{"a":1,"b":2,"c":3},{"a":1,"b":2,"c":3}]|
|[{"a":1,"b":2,"c":3}]                    |
|[]                                       |
+-----------------------------------------+



In [ ]:
df_02 = spark.read.load('/content/drive/MyDrive/Data/book2.csv', format='csv', sep=';', header=True)

In [ ]:
# df_02.printSchema()
df_02.show()

+---+----+--------------------+
| id|name|                json|
+---+----+--------------------+
|  1| aaa|{"a":1,"b": 2,"c"...|
|  2| bbb|{"a":1,"b": 2,"c"...|
|  3| ccc|{"a":1,"b": 2,"c"...|
+---+----+--------------------+



In [ ]:
schema = StructType([
                     StructField("a", IntegerType()),
                     StructField("b", IntegerType()),
                     StructField("c", IntegerType())
])

In [ ]:
df_02.select(from_json(df_02.json, schema)).show(5,False)

+---------------+
|from_json(json)|
+---------------+
|{1, 2, 3}      |
|{1, 2, 3}      |
|{1, 2, 3}      |
+---------------+



In [ ]:
df_02.select(from_json(df_02.json, schema).a).show()

+-----------------+
|from_json(json).a|
+-----------------+
|                1|
|                1|
|                1|
+-----------------+



In [ ]:
df_02.show(5, False)

+---+----+---------------------+
|id |name|json                 |
+---+----+---------------------+
|1  |aaa |{"a":1,"b": 2,"c": 3}|
|2  |bbb |{"a":1,"b": 2,"c": 3}|
|3  |ccc |{"a":1,"b": 2,"c": 3}|
+---+----+---------------------+



In [40]:
schema = ArrayType(StructType([
                     StructField("a", IntegerType()),
                     StructField("b", IntegerType()),
                     StructField("c", IntegerType())
]))
# schema = ArrayType(StringType())

In [41]:
df_02.select(from_json(df_02.json, schema)).show(5,False)


+---------------+
|from_json(json)|
+---------------+
|[{1, 2, 3}]    |
|[{1, 2, 3}]    |
|[{1, 2, 3}]    |
+---------------+



In [ ]:
df_02.coalesce(1).write.format('json').save('/content/drive/MyDrive/Data/book_out2.csv')

In [ ]:
df_02.select(df_02.id, df_02.name, from_json(df_02.json, schema).alias("json")).write.save('/content/drive/MyDrive/Data/book_out2', format='json')

In [ ]:
df_02.show(5, False)

+---+----+---------------------+
|id |name|json                 |
+---+----+---------------------+
|1  |aaa |{"a":1,"b": 2,"c": 3}|
|2  |bbb |{"a":1,"b": 2,"c": 3}|
|3  |ccc |{"a":1,"b": 2,"c": 3}|
+---+----+---------------------+



In [ ]:
# df_02.select(df_02.id, df_02.name, from_json(df_02.json, schema).alias("json")).rdd.map(lambda x:x["id", "name", "json"]).saveAsTextFile('/content/drive/MyDrive/Data/book_out2')
df_02.select(df_02.id, df_02.name, from_json(df_02.json, schema).alias("json")).rdd.map(lambda x:x["id", "name", "json"]).saveAsTextFile('/content/drive/MyDrive/Data/book_out2')

In [ ]:
# df_t = df_02.select(df_02.id, df_02.name, from_json(df_02.json, schema).alias("json"))
df_t = df_t.select(to_json(df_02.json).alias("json"))
  # .rdd.map(lambda x:x["id", "name", "json"]).collect()

In [ ]:
df_t.show()

+---+----+-----------+
| id|name|       json|
+---+----+-----------+
|  1| aaa|[{1, 2, 3}]|
|  2| bbb|[{1, 2, 3}]|
|  3| ccc|[{1, 2, 3}]|
+---+----+-----------+



In [ ]:
df=spark.createDataFrame([("a",1),("b",2)],["id","name"])

from pyspark.sql.functions import *

df.groupBy(lit("1")).\
agg(collect_list(struct(*[df.columns])).alias("cl")).\
select(to_json("cl").alias("jsn")).show(5, False)
# select(to_json("cl").alias("jsn")).\
# rdd.\
# map(lambda x:x["jsn"]).collect()


+-----------------------------------------+
|jsn                                      |
+-----------------------------------------+
|[{"id":"a","name":1},{"id":"b","name":2}]|
+-----------------------------------------+



In [ ]:
df2.toJSON().collect()

['{"name":{"firstname":"James","middlename":"","lastname":"Smith"},"id":"36636","gender":"M","salary":3100}',
 '{"name":{"firstname":"Michael","middlename":"Rose","lastname":""},"id":"40288","gender":"M","salary":4300}',
 '{"name":{"firstname":"Robert","middlename":"","lastname":"Williams"},"id":"42114","gender":"M","salary":1400}',
 '{"name":{"firstname":"Maria","middlename":"Anne","lastname":"Jones"},"id":"39192","gender":"F","salary":5500}',
 '{"name":{"firstname":"Jen","middlename":"Mary","lastname":"Brown"},"id":"","gender":"F","salary":-1}']

In [55]:
data = [1, """[{'a': 1, 'b': 'ro', 'c': 'has', 'd': '250'}, {'a': 1, 'b': 'ro', 'c': 'has', 'd': '250'}, {'a': 1, 'b': 'ro', 'c': 'has', 'd': '250'}]"""]

In [56]:
sch = ArrayType(StructType([
                  StructField("a", IntegerType()),
                  StructField("b", StringType()),
                  StructField("c", StringType()),
                  StructField("d", StringType()),
]))

In [57]:
d = spark.createDataFrame([data], ["id", "sch"])

In [58]:
d.show()

+---+--------------------+
| id|                 sch|
+---+--------------------+
|  1|[{'a': 1, 'b': 'r...|
+---+--------------------+



In [60]:
d.select(from_json(d.sch, sch)).show(5, False)

+---------------------------------------------------------+
|from_json(sch)                                           |
+---------------------------------------------------------+
|[{1, ro, has, 250}, {1, ro, has, 250}, {1, ro, has, 250}]|
+---------------------------------------------------------+

